# Step 1 - Scraping

### NASA Mars News

- Collect the latest News Title and Paragraph Text

In [ ]:
# Dependencies and setup
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import os
import requests
import pymongo

In [ ]:
# Set up splinter
executable_path = {"executable_path":ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless = False)

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [ ]:
# Define database and collection
db = client.mars_db
collection = db.headlines

In [ ]:
# URL of page to be scraped
newsURL = "https://redplanetscience.com"
browser.visit(newsURL)

In [ ]:
# Loop through homepage
for x in range(1):
    
    # HTML object
    html = browser.html
    
    # Parse HTML with BeautifulSoup
    soup = bs(html, "html.parser")
    
    # Retrieve all elements that contain News Titles and Paragraph Text
    articles = soup.find_all("div", class_ = "list_text")
    
    # Iterate through each headline
    for article in articles:
        
        # Use BeautifulSoup's find() method to navigate and retrieve attributes
        newsTitle = article.find("div", class_ = "content_title").text
        paragraphText = article.find("div", class_ = "article_teaser_body").text
        print("------------------------------------------")
        print(f"Article:  {newsTitle}")
        print(f"Content:  {paragraphText}")
    
    # Dictionary to be inserted as a MongoDB document
        post = {
            "Title": newsTitle,
            "Paragraph Content": paragraphText,
        }
            
        collection.insert_one(post)

### JPL Mars Space Images - Featured Image

- Find the image url for the current featured Mars image

In [ ]:
# URL of page to be scraped
imageURL = "https://spaceimages-mars.com/"
browser.visit(imageURL)

In [ ]:
# HTML object
imageHTML = browser.html
    
# Parse HTML with BeautifulSoup
imageSoup = bs(imageHTML, "html.parser")

In [ ]:
# Find href within anchor tag element
featuredImage = soup.find("a", class_ = "showimg fancybox-thumbs")["href"]

# Declare variable and combine main URL with link
featured_image_url = f"https://spaceimages-mars.com/{featuredImage}"
print(f"Featured Image URL:  {featured_image_url}")

In [ ]:
# Close remote browser
browser.quit()